In [4]:
import pickle
import pandas as pd
import numpy as np
import datetime as dt
from sklearn import preprocessing 
from xgboost.sklearn import XGBRegressor
from sklearn import metrics   #Additional     scklearn functions
from sklearn.model_selection import GridSearchCV
import xgboost
from lightgbm import LGBMRegressor
from sklearn.metrics import make_scorer # define our own error function
import time
import warnings
warnings.filterwarnings("ignore")

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [5]:
X_train_ = np.load('X_train.npz')
X_test_ = np.load('X_test.npz')
Y_train_ = np.load('Y_train.npz')

X_train = X_train_['arr_0'] 

X_test = X_test_['arr_0']
Y_train = Y_train_['arr_0']

# train_pre = preprocessing.normalize(np.concatenate([X_train,X_test]))
# X_train_pre = train_pre[:47500,:]
# X_test_pre = train_pre[47500:,:]


# print(train_pre.shape)

In [6]:
Y_data_p = Y_train[:,0]
Y_data_m = Y_train[:,1]
Y_data_a = Y_train[:,2]

In [7]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(47500, 10000)
(47500, 3)
(2500, 10000)


### Origin data

In [8]:
data_origin_pre = preprocessing.normalize(np.concatenate([X_train,X_test]),axis = 0)

In [9]:
train_Xorigin_pre = data_origin_pre[:47500,:]
test_Xorigin_pre = data_origin_pre[47500:,:]

### Feature Selection

In [7]:
data = np.concatenate([X_train,X_test])

In [8]:
data.shape

(50000, 10000)

In [9]:
VAC = data[:,5000:]
MSD = data[:,:5000]

In [10]:
class MSD_Line:
    def __init__(self,MSD):
        self.y = MSD
        self.slope = None
        self.bias = MSD[:,0].reshape((-1,1))
        self.x = np.array([i for i in range(5000)]*MSD.shape[0]).reshape((MSD.shape[0],-1))
        self.error = None
        self.error_abs = None
        self.error_square = None
        
    def get_MSDLine(self):
        self.slope = (self.y[:,4999] - self.y[:,0])/ 5000
        self.slope = self.slope.reshape((-1,1))
    def get_error(self):
        y = self.slope*self.x + self.bias
        self.error = self.y - y
        self.error_abs = np.abs(self.error)
        self.error_square = np.square(self.error)

In [11]:
msd_Line = MSD_Line(MSD)
msd_Line.get_MSDLine()
msd_Line.get_error()
msd_error = msd_Line.error
msd_error_abs = msd_Line.error_abs
msd_error_square = msd_Line.error_square

In [168]:
msd_error.shape

(50000, 5000)

In [14]:
vac_all = []
for j in range(VAC.shape[0]):
    vac_ways = []
    for i in range(0,5000,100):
        vac = VAC[j,i:i+100]
        vac_ways.append(vac)
    vac_all.append(vac_ways)
vac_all = np.array(vac_all)

In [15]:
vac_all.shape

(50000, 50, 100)

In [163]:
def filter_vac(vac_all,threshold = 0.1):
    converge_all = []
    unders_all = []
    mins_all = []
    
    # each data
    for k,vac in enumerate(vac_all):
        converge_vac = []
        unders_vac = []
        mins_vac = []
        
        # each volcity
        for v,bb_ in enumerate(vac):
            flag1 = 0
            flag2 = 0
            flag3 = 0
            
            # each time interval
            for i in range(100):
                
                # cross 0
                if (bb_[i]<0.0 and flag1 == 0):
                    flag1 = 1
                    unders_vac.append(i)
                    
                # arrive min
                if(i != 0 and i!= 99 and bb_[i-1]<bb_[i] and bb_[i]<bb_[i+1] and flag2 == 0):
                    flag2 = 1
                    mins_vac.append(i)
                    
                # converge under 0.1
                if(abs(bb_[i])<threshold and flag2==1):
                    flag3 = 1
                    converge_vac.append(i) 
                    break
                    
            try:
                assert(flag1 == 1)
            except:
#                 print('Not under',k,v)
                unders_vac.append(100)
            try:
                assert(flag2 == 1)
            except:
#                 print('Not min',k,v)
                mins_vac.append(100)
            try:
                assert(flag3 == 1)
            except:
#                 print('Not converge',k,v)
                converge_vac.append(100) 
                
        converge_all.append(converge_vac)
        unders_all.append(unders_vac)
        mins_all.append(mins_vac)
        
    converge_all,unders_all,mins_all = np.array(converge_all),np.array(unders_all),np.array(mins_all)
    
    return converge_all,unders_all,mins_all

In [164]:
converge_all_vac,unders_all_vac,mins_all_vac = filter_vac(vac_all)

In [177]:
new_VAC = np.concatenate([VAC,unders_all_vac,mins_all_vac,converge_all_vac],axis = 1)

In [178]:
new_VAC.shape

(50000, 5150)

In [202]:
new_data = np.concatenate([msd_error,new_VAC],axis = 1)

In [203]:
new_data.shape

(50000, 10150)

In [204]:
new_data_pre = preprocessing.normalize(new_data,axis = 0)

In [214]:
cv_pre = new_data_pre[:47500,:]

In [210]:
test_length = 2500
train_length = int(47500*0.8)
train_pre =  new_data_pre[:train_length,:]
val_pre = new_data_pre[train_length:-test_length,:]
test_pre = new_data_pre[-test_length:,:]

Y_train_p = Y_data_p[:train_length]
Y_train_m = Y_data_m[:train_length]
Y_train_a = Y_data_a[:train_length]
Y_val_p = Y_data_p[train_length:]
Y_val_m = Y_data_m[train_length:]
Y_val_a = Y_data_m[train_length:]

In [216]:
def NAE(y_true, y_pred):
    diff = np.abs(y_true - y_pred)
    n_diff = diff/y_true
    return n_diff

In [217]:
NAE_score = make_scorer(NAE, greater_is_better=False)

* w is 300 for penetration_rate, 1 for mesh_size and 200 for alpha
* value_1 is the penatration rate 
* value_2 is the mesh_size 
* value_3 is the alpha

### Alpha fine tune

In [10]:
gbm_a = LGBMRegressor(boosting_type='gbdt',
                         objective = 'regression',
                         metric = 'l1',
                         verbose = 0,
                         learning_rate = 0.005, # 可與num_iterations做等比縮放
                         num_leaves = 35, # leave數目，通常要< 2^max_depth
                         min_data_in_leaf = 200, # leaf內的資料數目
                         feature_fraction=0.8, # feature使用數量
                         bagging_fraction= 0.9,  # bagging時resample的數目
                         bagging_freq= 8, #多久進行一次bagging
                         lambda_l1= 0.6, # l1 懲罰係數
                         lambda_l2= 0,
                         max_depth = 10,
                        num_iterations = 3000,
                         )

In [223]:
start = time.time()
param_test1_a = {'num_iterations':[i for i in range(100,1000,100)]}
gsearch1_a = GridSearchCV(estimator = gbm_a, 
                       param_grid = param_test1_a, scoring='neg_mean_absolute_error',cv=5,n_jobs = 6)
gsearch1_a.fit(cv_pre,Y_data_a)
end = time.time()
print((end-start)/60)
print(gsearch1_a.cv_results_)
print(gsearch1_a.best_params_)
print(gsearch1_a.best_score_)

452.57876824537914
{'mean_fit_time': array([ 869.2028986 , 1090.82451406, 1416.08023572, 1610.2939446 ,
       2013.39825139, 2451.4418879 , 2993.97435231, 3645.72301188,
       4204.14466214]), 'std_fit_time': array([ 21.56055578,  30.08252949,  53.26791121,  50.04603906,
        39.61327496,  32.63182097,  82.34732854,  74.95080164,
       181.76828266]), 'mean_score_time': array([2.12584949, 1.51142898, 2.24298196, 2.60484853, 2.64033809,
       2.7888732 , 3.29634981, 3.84104657, 3.82908502]), 'std_score_time': array([0.24050185, 0.49726659, 0.76059525, 0.66895987, 0.61332953,
       1.06970571, 0.23820685, 1.07555465, 0.56486188]), 'param_num_iterations': masked_array(data=[100, 200, 300, 400, 500, 600, 700, 800, 900],
             mask=[False, False, False, False, False, False, False, False,
                   False],
       fill_value='?',
            dtype=object), 'params': [{'num_iterations': 100}, {'num_iterations': 200}, {'num_iterations': 300}, {'num_iterations': 400}, {'n

In [257]:
start = time.time()
param_test1_a = {'num_iterations':[i for i in [900]]}
gsearch1_a = GridSearchCV(estimator = gbm_a, 
                       param_grid = param_test1_a, scoring='neg_mean_absolute_error',cv=5,n_jobs = 6)
gsearch1_a.fit(train_Xorigin_pre,Y_data_a)
end = time.time()
print((end-start)/60)
print(gsearch1_a.cv_results_)
print(gsearch1_a.best_params_)
print(gsearch1_a.best_score_)

116.18145705461502
{'mean_fit_time': array([3649.88862901]), 'std_fit_time': array([63.28939]), 'mean_score_time': array([3.17332506]), 'std_score_time': array([0.29223112]), 'param_num_iterations': masked_array(data=[900],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'num_iterations': 900}], 'split0_test_score': array([-0.00723314]), 'split1_test_score': array([-0.00708408]), 'split2_test_score': array([-0.00709229]), 'split3_test_score': array([-0.00712336]), 'split4_test_score': array([-0.0070677]), 'mean_test_score': array([-0.00712011]), 'std_test_score': array([5.93381453e-05]), 'rank_test_score': array([1], dtype=int32), 'split0_train_score': array([-0.00652816]), 'split1_train_score': array([-0.00654503]), 'split2_train_score': array([-0.00653899]), 'split3_train_score': array([-0.006527]), 'split4_train_score': array([-0.0065512]), 'mean_train_score': array([-0.00653808]), 'std_train_score': array([9.40594227e-06])}
{'num_iterations'

In [231]:
# start = time.time()
# param_test2_a = {'max_depth':[i for i in range(6,21,2)],'num_leaves':[35,50,75]}
# gsearch2_a = GridSearchCV(estimator = gbm_a,param_grid = param_test2_a, 
#                           scoring='neg_mean_absolute_error', cv=5, n_jobs = 6)
# gsearch2_a.fit(cv_pre,Y_data_a)
# end = time.time()
# gsearch2_a.cv_results_, gsearch2_a.best_params_, gsearch2_a.best_score_
print((end-start)/60)
print(gsearch2_a.cv_results_)
print(gsearch2_a.best_params_)
print(gsearch2_a.best_score_)

334.9988782008489
{'mean_fit_time': array([1163.19136076,  744.98129535,  550.427948  ,  523.73808599,
        535.92000623,  552.64672279,  808.08686204, 1186.06347785,
       1203.37447257, 1208.31289482, 1181.21912556, 1191.92829018,
       1164.31797724,  853.96167035,  508.88222632,  627.36306977,
        919.56846991,  954.82521925,  918.91958904,  972.19309258,
        930.92700019,  949.47568722,  945.79761353,  901.9609108 ]), 'std_fit_time': array([ 13.13309983, 228.03714411,  63.26363703,  20.97268285,
        18.18733483,  22.69141113, 109.18493493, 107.35526141,
        45.20894201,  28.8507291 ,  29.99410976,  38.08488349,
        43.22292694, 184.45961677,  62.15673231, 180.00499379,
        48.90482097,  23.77989689,  12.79839552,  29.67256455,
        17.79546623,  16.12036575,  30.47328042,  30.53636031]), 'mean_score_time': array([1.59603701, 1.29577866, 0.9857367 , 1.10863256, 1.01032557,
       0.86171646, 2.48654051, 2.46576486, 2.4492568 , 2.39883618,
       2.14

In [ ]:
gbm_a.fit(train_Xorigin_pre,Y_data_a)
prediction_a = gbm_a.predict(test_Xorigin_pre)

### Mesh_size fine tune

In [11]:
gbm_m = LGBMRegressor(boosting_type='gbdt',
                         objective = 'regression',
                         metric = 'l1',
                         verbose = 0,
                         learning_rate = 0.005, # 可與num_iterations做等比縮放
                         num_leaves = 35, # leave數目，通常要< 2^max_depth
                         min_data_in_leaf = 200, # leaf內的資料數目
                         feature_fraction=0.8, # feature使用數量
                         bagging_fraction= 0.9,  # bagging時resample的數目
                         bagging_freq= 8, #多久進行一次bagging
                         lambda_l1= 0.6, # l1 懲罰係數
                         lambda_l2= 0,
                         max_depth = 10,
                         num_iterations = 3000,
                         )

In [244]:
start = time.time()
param_test1_m = {'num_iterations':[i for i in [900]]}
gsearch1_m = GridSearchCV(estimator = gbm_m, 
                       param_grid = param_test1_m, scoring='neg_mean_absolute_error',cv=5,n_jobs = 8)
gsearch1_m.fit(cv_pre,Y_data_m)
end = time.time()
gsearch1_m.cv_results_, gsearch1_m.best_params_, gsearch1_m.best_score_
print((end-start)/60)
print(gsearch1_m.cv_results_)
print(gsearch1_m.best_params_)
print(gsearch1_m.best_score_)

186.44594598611195
{'mean_fit_time': array([5018.12359314]), 'std_fit_time': array([27.60263105]), 'mean_score_time': array([3.34723992]), 'std_score_time': array([0.37789898]), 'param_num_iterations': masked_array(data=[900],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'num_iterations': 900}], 'split0_test_score': array([-27.13142174]), 'split1_test_score': array([-27.3071136]), 'split2_test_score': array([-27.19054919]), 'split3_test_score': array([-26.86024882]), 'split4_test_score': array([-27.69625661]), 'mean_test_score': array([-27.23711799]), 'std_test_score': array([0.27242554]), 'rank_test_score': array([1], dtype=int32), 'split0_train_score': array([-24.56256406]), 'split1_train_score': array([-24.5278059]), 'split2_train_score': array([-24.57935206]), 'split3_train_score': array([-24.65706878]), 'split4_train_score': array([-24.48692304]), 'mean_train_score': array([-24.56274277]), 'std_train_score': array([0.05681991])}
{'num_it

In [255]:
start = time.time()
param_test1_m = {'num_iterations':[i for i in [900]]}
gsearch1_m = GridSearchCV(estimator = gbm_m, 
                       param_grid = param_test1_m, scoring='neg_mean_absolute_error',cv=5,n_jobs = 8)
gsearch1_m.fit(train_Xorigin_pre,Y_data_m)
end = time.time()
gsearch1_m.cv_results_, gsearch1_m.best_params_, gsearch1_m.best_score_
print((end-start)/60)
print(gsearch1_m.cv_results_)
print(gsearch1_m.best_params_)
print(gsearch1_m.best_score_)

116.18145705461502
{'mean_fit_time': array([4735.14412012]), 'std_fit_time': array([135.54886165]), 'mean_score_time': array([2.72758183]), 'std_score_time': array([0.20255594]), 'param_num_iterations': masked_array(data=[900],
             mask=[False],
       fill_value='?',
            dtype=object), 'params': [{'num_iterations': 900}], 'split0_test_score': array([-20.17050263]), 'split1_test_score': array([-20.30441569]), 'split2_test_score': array([-20.18485149]), 'split3_test_score': array([-20.02469643]), 'split4_test_score': array([-20.36514057]), 'mean_test_score': array([-20.20992136]), 'std_test_score': array([0.11790911]), 'rank_test_score': array([1], dtype=int32), 'split0_train_score': array([-18.02923725]), 'split1_train_score': array([-18.06056093]), 'split2_train_score': array([-18.01411148]), 'split3_train_score': array([-18.09564756]), 'split4_train_score': array([-18.07075045]), 'mean_train_score': array([-18.05406154]), 'std_train_score': array([0.0291973])}
{'num_

In [ ]:
param_test2_m = {'max_depth':[i for i in range(5,21,5)], 'min_samples_split':[i for i in range(20,1001,100)]}
gsearch2_m = GridSearchCV(estimator = LGBMRegressor(learning_rate=0.1, n_estimators=900, min_samples_leaf=20, 
      max_features='sqrt', subsample=0.8, random_state=10), 
   param_grid = param_test2_m, scoring='neg_mean_absolute_error', cv=5, n_jobs = 6)
gsearch2_m.fit(train_Xorigin_pre,Y_data_m)
gsearch2_m.cv_results_, gsearch2_m.best_params_, gsearch2_m.best_score_

In [ ]:
gbm_m.fit(train_Xorigin_pre,Y_data_m)
prediction_m = gbm_m.predict(test_Xorigin_pre)

### Penetration_rate fine tune

In [12]:
gbm_p = LGBMRegressor(boosting_type='gbdt',
                         objective = 'regression',
                         metric = 'l1',
                         verbose = 0,
                         learning_rate = 0.005, # 可與num_iterations做等比縮放
                         num_leaves = 35, # leave數目，通常要< 2^max_depth
                         min_data_in_leaf = 200, # leaf內的資料數目
                         feature_fraction=0.8, # feature使用數量
                         bagging_fraction= 0.9,  # bagging時resample的數目
                         bagging_freq= 8, #多久進行一次bagging
                         lambda_l1= 0.6, # l1 懲罰係數
                         lambda_l2= 0,
                         max_depth = 10,
                          num_iterations = 3000,
                         )

In [ ]:
# for origin data
start = time.time()
param_test1_p = {'num_iterations':[i for i in [1500]]}
gsearch1_p = GridSearchCV(estimator = gbm_p,
                       param_grid = param_test1_p, scoring='neg_mean_absolute_error',cv=3,n_jobs = 8)
gsearch1_p.fit(train_Xorigin_pre,Y_data_p)
end = time.time()
print((end-start)/60/60)
gsearch1_p.cv_results_, gsearch1_p.best_params_, gsearch1_p.best_score_

In [270]:
print((end-start)/60/60)

8.518781208528413


In [271]:
gsearch1_p.cv_results_, gsearch1_p.best_params_, gsearch1_p.best_score_

({'mean_fit_time': array([24449.36764757]),
  'std_fit_time': array([448.28178627]),
  'mean_score_time': array([24.8464818]),
  'std_score_time': array([0.56513128]),
  'param_num_iterations': masked_array(data=[9000],
               mask=[False],
         fill_value='?',
              dtype=object),
  'params': [{'num_iterations': 9000}],
  'split0_test_score': array([-0.09841822]),
  'split1_test_score': array([-0.09929468]),
  'split2_test_score': array([-0.09816657]),
  'mean_test_score': array([-0.09862649]),
  'std_test_score': array([0.00048352]),
  'rank_test_score': array([1], dtype=int32),
  'split0_train_score': array([-0.08682431]),
  'split1_train_score': array([-0.08592798]),
  'split2_train_score': array([-0.08670103]),
  'mean_train_score': array([-0.08648444]),
  'std_train_score': array([0.00039668])},
 {'num_iterations': 9000},
 -0.09862648543267052)

In [ ]:
gbm_p.fit(train_Xorigin_pre,Y_data_p)
prediction_p = gbm_p.predict(test_Xorigin_pre)

### Prediction

In [ ]:
def output(outputfile,pre_a,pre_m,pre_p):
    df = pd.DataFrame(np.concatenate([pre_p.reshape((-1,1)),pre_m.reshape((-1,1)),pre_a.reshape((-1,1))],axis=1))
    df.to_csv(outputfile,index=False,header=False)

In [ ]:
output('lightgbm_3000_005.csv',prediction_a,prediction_m,prediction_p)